# Image-based Deepfake Detection using Xception
This notebook extracts a single representative frame on-the-fly from video files and classifies them as real or fake using a CNN (Xception) model.

In [ ]:

import os
import cv2
import random
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# GPU memory growth setup
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
        print(e)


In [ ]:

real_paths = [
    "./dataset/original_sequences/actors/raw/videos",
    "./dataset/original_sequences/youtube/raw/videos"
]

fake_paths = [
    "./dataset/manipulated_sequences/DeepFakeDetection/raw/videos",
    "./dataset/manipulated_sequences/Deepfakes/raw/videos",
    "./dataset/manipulated_sequences/Face2Face/raw/videos",
    "./dataset/manipulated_sequences/FaceSwap/raw/videos",
    "./dataset/manipulated_sequences/NeuralTextures/raw/videos",
    "./dataset/manipulated_sequences/FaceShifter/raw/videos"
]

def list_videos(paths):
    videos = []
    for path in paths:
        videos += [os.path.join(path, f) for f in os.listdir(path) if f.endswith(".mp4")]
    return videos

real_videos = list_videos(real_paths)
fake_videos = list_videos(fake_paths)
real_labels = [0] * len(real_videos)
fake_labels = [1] * len(fake_videos)

all_videos = real_videos + fake_videos
all_labels = real_labels + fake_labels

train_videos, test_videos, train_labels, test_labels = train_test_split(all_videos, all_labels, test_size=0.2, stratify=all_labels, random_state=42)


In [ ]:

def extract_single_frame(video_path, resize=(160, 160)):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    middle_frame = total_frames // 2
    cap.set(cv2.CAP_PROP_POS_FRAMES, middle_frame)
    ret, frame = cap.read()
    cap.release()
    if ret:
        frame = cv2.resize(frame, resize)
        frame = frame.astype(np.float32) / 255.0
        return frame
    else:
        return np.zeros((*resize, 3), dtype=np.float32)


In [ ]:

IMG_SIZE = (160, 160)
BATCH = 32
AUTOTUNE = tf.data.AUTOTUNE

def frame_generator(video_list, label_list):
    for path, label in zip(video_list, label_list):
        frame = extract_single_frame(path, resize=IMG_SIZE)
        yield frame, label

def create_dataset(video_list, label_list):
    dataset = tf.data.Dataset.from_generator(
        lambda: frame_generator(video_list, label_list),
        output_types=(tf.float32, tf.int32),
        output_shapes=((IMG_SIZE[0], IMG_SIZE[1], 3), ())
    )
    return dataset.shuffle(1000).batch(BATCH).prefetch(AUTOTUNE)

train_ds = create_dataset(train_videos, train_labels)
test_ds = create_dataset(test_videos, test_labels)


In [ ]:
os.makedirs("saved_model", exist_ok=True)
checkpoint_path = "saved_model/best_model.h5"

In [ ]:

base_model = Xception(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model.trainable = False

inputs = Input(shape=(*IMG_SIZE, 3))
x = tf.keras.applications.xception.preprocess_input(inputs)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(1, activation='sigmoid')(x)

model = Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-4), loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()


In [ ]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2),
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True, verbose=1)
]

history = model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=20,
    callbacks=callbacks
)


In [ ]:
model.save("saved_model/final_model.keras")


In [ ]:
import matplotlib.pyplot as plt

def plot_metrics(history):
    metrics = ['accuracy', 'loss', 'precision', 'recall']
    for metric in metrics:
        plt.figure()
        plt.plot(history.history[metric], label='train_' + metric)
        plt.plot(history.history['val_' + metric], label='val_' + metric)
        plt.title(f'{metric.capitalize()} Curve')
        plt.xlabel('Epochs')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.grid(True)
        plt.show()

plot_metrics(history)


In [ ]:

y_true = np.concatenate([y for _, y in test_ds])
y_pred = (model.predict(test_ds) > 0.5).astype(int).ravel()
print(classification_report(y_true, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt

# Get one sample from test set
for img_batch, label_batch in test_ds.take(1):
    image = img_batch[0].numpy()
    label = label_batch[0].numpy()

# LIME expects function that returns probabilities
def predict_fn(images):
    return model.predict(np.array(images))

explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(
    image.astype('double'),
    predict_fn,
    top_labels=2,
    hide_color=0,
    num_samples=1000
)

temp, mask = explanation.get_image_and_mask(
    label=int(label),
    positive_only=True,
    num_features=10,
    hide_rest=False
)

plt.imshow(mark_boundaries(temp, mask))
plt.title(f"LIME Explanation - Label: {int(label)}")
plt.axis('off')
plt.show()
